In [1]:
print("Zero-shot medical MT")

Zero-shot medical MT


In [2]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 6.4 MB/s eta 0:00:00


In [3]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
Tesla P100-PCIE-16GB


In [4]:
import transformers
transformers.utils.logging.set_verbosity_error()

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "Qwen/Qwen3-0.6B"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

model.eval()

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

...

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

model.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 1024)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=1024, out_features=2048, bias=False)
          (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (up_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (down_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((1024,), eps=1e-06)
        (post_attention_layernorm): Qwe

In [6]:
prompt = "Hello, how are you?"

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=50
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Hello, how are you? I'm currently in the middle of a study, but I'm still working on it. I'm not sure if I'm doing well in my studies. I'm looking for help with my studies. I'm not sure how to start. I need


In [7]:
def medical_translation_prompt(src_text):
    return f"""
### Instruction:
Translate the following medical sentence from English to Vietnamese.

### Constraints:
- Medical domain translation
- Keep all information
- No explanation
- Vietnamese only

### English:
{src_text}

### Vietnamese:
"""

In [8]:
def translate(text, max_new_tokens=256):
    prompt = medical_translation_prompt(text)
    
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False
        )
    
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return decoded, decoded.split("### Vietnamese:")[-1].split("\n\n")[0].strip()

In [9]:
test_sentence = (
    "Knowledge, practices in public health service utilization among health insurance card’s holders "
    "and influencing factors in Vientiane, Lao"
)

origin_translated, processed_translated = translate(test_sentence)
# print(origin_translated)
print(processed_translated)

Các bệnh, các hoạt động trong chăm sóc sức khỏe tại các cơ sở y tế công cộng của người sử dụng thẻ bảo hiểm y tế và các yếu tố ảnh hưởng đến Vientiane, Lao


In [10]:
def read_pair_files(src_file_path, trg_file_path):
    print(f"Reading data from: {src_file_path} and {trg_file_path}...")

    with open(src_file_path, 'r', encoding='utf-8') as f:
        src_lines = f.read().strip().split('\n')

    with open(trg_file_path, 'r', encoding='utf-8') as f:
        trg_lines = f.read().strip().split('\n')

    # Check equal row src and dst
    assert len(src_lines) == len(trg_lines), "Error: Rows of source and target are difference"

    dataset = []
    for src, trg in zip(src_lines, trg_lines):
        dataset.append({'src': src.strip(), 'ref': trg.strip()})

    print(f"Done read: {len(dataset)} rows.")
    return dataset

In [11]:
data = read_pair_files(
    src_file_path="/kaggle/input/vlst-medical-mt-cleaned/clean_public_test.en.txt",
    trg_file_path="/kaggle/input/vlst-medical-mt-cleaned/clean_public_test.vi.txt",
)

Reading data from: /kaggle/input/vlst-medical-mt-cleaned/clean_public_test.en.txt and /kaggle/input/vlst-medical-mt-cleaned/clean_public_test.vi.txt...
Done read: 3000 rows.


In [12]:
data[:5]

[{'src': 'Knowledge, practices in public health service utilization among health insurance card’s holders and influencing factors in Vientiane, Lao',
  'ref': 'Thực trạng kiến thức và thực hành của người có thẻ bảo hiểm y tế trong sử dụng dịch vụ khám chữa bệnh ở các cơ sở y tế công và một số yếu tố ảnh hưởng tại tỉnh Viêng Chăn, CHDCND Lào, năm 2017'},
 {'src': "Describe knowledge, practices in public health service utilization among health insurance card's holders and influencing factors in Vientiane, Lao PDR, 2017.",
  'ref': 'Mô tả thực trạng kiến thức, thực hành của người có thẻ bảo hiểm y tế trong sử dụng dịch vụ khám chữa bệnh ở các cơ sở y tế công và một số yếu tố liên quan tại tỉnh Viêng Chăn, Cộng hoà Dân chủ Nhân dân Lào năm 2017.'},
 {'src': "Methodology: A cross sectional study was used among 928 adult health insurance card's holders in Phone Hong and Keo Oudom districts, Vientiane province.",
  'ref': 'Phương pháp: Thiết kế nghiên mô tả cắt ngang được thực hiện trên 928 n

In [13]:
from tqdm import tqdm

In [14]:
preds = []
refs = []

for item in tqdm(data):
    _, pred = translate(item["src"])
    preds.append(pred)
    refs.append([item["ref"]])
    
    if (len(pred) <= 0):
        print("-" * 50)
        print("EMPTY PREDICT")
        print("SRC :", item["src"])
        print("PRED:", pred)
        print("REF :", item["ref"])
        print("-" * 50)

 52%|█████▏    | 1549/3000 [4:04:09<3:46:29,  9.37s/it]

--------------------------------------------------
EMPTY PREDICT
SRC : SURVEY OF CLINICAL CHARACTERISTICS, LABORATORY FNDINGS AND CONTINUOUS RENAL REPLACEMENT THERAPY IN POSTPARTUMS WITH ACUTE KIDNEY INJURY Aim of survey is to study the clinical characterisitics, laboratory findings and initial evaluation of continuous renal replacement therapy in postpartum patients with acute renal injujy.
PRED: 
REF : NGHIÊN CỨU ĐẶC ĐIỂM LÂM SÀNG, CẬN LÂM SÀNG TÓM TẮT Mục tiêu nghiên cứu nhằm khảo sát đặc điểm lâm sàng, cận lâm sàng và bước đầu đánh giá hiệu quả của lọc máu liên tục ở bệnh nhân hậu sản có tổn thương thận cấp.
--------------------------------------------------


 87%|████████▋ | 2623/3000 [6:52:52<59:22,  9.45s/it]

--------------------------------------------------
EMPTY PREDICT
SRC : Environment - working conditions and MSDs on nurses The working environment of health workers in general and nurses in particular has many potential risk factors that greatly affect the prevalence and incidence of MSDs, including: - Physical risk factors / postures: care work, transportation of patients, heavy objects, wrong posture... - Extended working time: long working time, duty, shift work lasts more than 12 hours, night work, holiday work... 5 - Unreasonable organization and workload, lack of social support, jobs requiring high responsibility... - Toxic and noisy working environment... 1.3.
PRED: 
REF : Môi trường – điều kiện làm việc và tình trạng rối loạn cơ xương trên điều dưỡng viên Môi trường làm việc của nhân viên y tế nói chung và ĐDV nói riêng tiềm ẩn nhiều yếu tố nguy cơ ảnh hưởng lớn đến tỉ lệ mắc RLCX, bao gồm: - Yếu tố nguy cơ vật lý / tư thế: công việc chăm sóc bệnh nhân, vận chuyển bệnh nhân, ma

100%|██████████| 3000/3000 [7:52:19<00:00,  9.45s/it]


In [15]:
preds[:5]

['Các bệnh, các hoạt động trong chăm sóc sức khỏe tại các cơ sở y tế công cộng của người sử dụng thẻ bảo hiểm y tế và các yếu tố ảnh hưởng đến Vientiane, Lao',
 '**Kết quả của việc sử dụng dịch vụ chăm sóc sức khỏe trong cộng đồng tại Vientiane, Lao PDR, năm 2017.**',
 '**Mô hình nghiên cứu:** Một nghiên cứu chi tiết được thực hiện trong 928 người sử dụng thẻ bảo hiểm y tế tại các khu vực Phone Hong và Keo Oudom, Vientiane province.',
 'Tổng số người có thẻ trong cơ sở y tế được biết đến với việc sử dụng dịch vụ y tế miễn phí là 44,5% và được thông báo về thông tin bảo hiểm y tế là 34,8%.',
 'Tỷ lệ người dùng thẻ card tại các cơ sở y tế được đăng ký đầu tiên đạt 61.8%.']

In [16]:
refs[:5]

[['Thực trạng kiến thức và thực hành của người có thẻ bảo hiểm y tế trong sử dụng dịch vụ khám chữa bệnh ở các cơ sở y tế công và một số yếu tố ảnh hưởng tại tỉnh Viêng Chăn, CHDCND Lào, năm 2017'],
 ['Mô tả thực trạng kiến thức, thực hành của người có thẻ bảo hiểm y tế trong sử dụng dịch vụ khám chữa bệnh ở các cơ sở y tế công và một số yếu tố liên quan tại tỉnh Viêng Chăn, Cộng hoà Dân chủ Nhân dân Lào năm 2017.'],
 ['Phương pháp: Thiết kế nghiên mô tả cắt ngang được thực hiện trên 928 người trưởng thành có thẻ bảo hiểm y tế tại 2 huyện Phone Hong và Keo Oudom, tỉnh Viêng Chăn.'],
 ['Kết quả: Tỷ lệ người biết được khám chữa bệnh (KCB) miễn phí tại nơi đăng ký ban đầu chiếm 44,5%, được cung cấp thông tin về bảo hiểm y tế (BHYT) chiếm 34,8%.'],
 ['Tỷ lệ người có thẻ BHYT thực hành khám chữa bệnh đúng nơi đăng ký KCB ban đầu chiếm 61,8%.']]

In [17]:
import json

with open("zs_predictions.json", 'w', encoding='utf-8') as f:
    json.dump(preds, f, ensure_ascii=False, indent=4)

In [ ]:
import sacrebleu

bleu = sacrebleu.corpus_bleu(preds, [refs])
print("BLEU baseline:", bleu.score)

BLEU baseline: 11.623694797200967
